# COGS 108 - EDA Checkpoint

# Names

- Ant Man
- Hulk
- Iron Man
- Thor
- Wasp

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Setup

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None #suppressing SettingWithCopyWarning: used to not set a value

# Data Cleaning

In [2]:
years = ['2012','2013','2014','2015','2016','2017',
'2018','2019','2020','2021','2022'] # years to be observed

#reading csv
weo = pd.read_csv('WEO.csv')
happiness = pd.read_csv('Happiness.csv')

#using set to extract common names
weo_country = set(weo["Country"].unique().tolist())
happiness_country = set(happiness["Country name"].unique().tolist())

common_country = weo_country & happiness_country
weo_not = weo_country - common_country
happiness_not = happiness_country - common_country

print(sorted(happiness_not))
print(sorted(weo_not))

['Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cuba', 'Czechia', 'Gambia', 'Hong Kong S.A.R. of China', 'Iran', 'Ivory Coast', 'Kyrgyzstan', 'Laos', 'Slovakia', 'Somaliland region', 'South Korea', 'State of Palestine', 'Syria', 'Turkiye']
['Andorra', 'Antigua and Barbuda', 'Aruba', 'Barbados', 'Brunei Darussalam', 'Cabo Verde', 'Czech Republic', "CÙte d'Ivoire", 'Democratic Republic of the Congo', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Fiji', 'Grenada', 'Guinea-Bissau', 'Hong Kong SAR', 'Islamic Republic of Iran', 'Kiribati', 'Korea', 'Kyrgyz Republic', 'Lao P.D.R.', 'Macao SAR', 'Marshall Islands', 'Micronesia', 'Nauru', 'Palau', 'Papua New Guinea', 'Puerto Rico', 'Republic of Congo', 'Samoa', 'San Marino', 'Seychelles', 'Slovak Republic', 'Solomon Islands', 'St. Kitts and Nevis', 'St. Lucia', 'St. Vincent and the Grenadines', 'S„o TomÈ and PrÌncipe', 'The Bahamas', 'The Gambia', 'Timor-Leste', 'Tonga', 'Tuvalu', 'T¸rkiye', 'Vanuatu', 'West Bank and Gaza']


Above is the lists of countries that are not common from both sets of data. 
First is from the Happiness Report and second is from the WEO.

Since some country names are reported different in each of the organizations, we manually matched such countries: 

* Congo (Brazzaville) -> Republic of Congo
* Congo (Kinshasa) -> Democratic Republic of the Congo
* Czechia -> Czech Republic
* Hong Kong S.A.R of China -> Hong Kong SAR
* Iran -> Iran (from Islamic Republic of Iran)
* Kyrgyzstan -> Kyrgyzstan (from Kyrgyz Republic)
* Laos -> Laos (from Lao P.D.R)
* Slovakia -> Slovakia (from Slovak Republic)
* South Korea -> South Korea (from Korea)
* Turkiye -> Turkiye (from T¸rkiye)
* Cuba, Gambia, Ivory Coast, Somaliland region, State of Palestine, Syria - > removed

In [3]:
#replacing left to right
happiness['Country name'].replace(['Congo (Brazzaville)', 'Congo (Kinshasa)', 'Czechia', 'Hong Kong S.A.R. of China'], 
            ['Republic of Congo', 'Democratic Republic of the Congo', 'Czech Republic', 'Hong Kong SAR'], inplace = True)
#replacing right to (from [name])
weo['Country'].replace(['Islamic Republic of Iran', 'Kyrgyz Republic', 'Lao P.D.R.', 'Slovak Republic', 'Korea', 'T¸rkiye'],
                 ['Iran', 'Kyrgyzstan', 'Laos', 'Slovakia', 'South Korea', 'Turkiye'], inplace = True)

#checking
weo_country = set(weo["Country"].unique().tolist())
happiness_country = set(happiness["Country name"].unique().tolist())
common_country = list(sorted(weo_country & happiness_country))
print(common_country)

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong SAR', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania', 'Luxembou

The above is the list of countries that are common from both of the lists. 

Note that these countries may not have all entries for variables we are observing and are subjected to be excluded in the later process.

From here on, a single data frame: `data` containing entries from both dataframe will be constructed.

In [4]:
#creating unified data frame: data

#dropping other variables
data = happiness[['Country name', 'year', 'Life Ladder', 'Positive affect', 'Negative affect']] 
#only keeping those after 2012
data.drop(data[data['year'] < 2012].index, inplace = True) 
#numerical year value is no longer needed / potential to influence other num vals
data['year'] = data['year'].apply(str) 

#adding weo variables to data
weo_vars = weo['Subject Descriptor'].unique().tolist()
for var in weo_vars:
    data[var] = np.nan
    
#copying of weo values to data
weo = weo.drop(columns = ['Subject Notes','Units', 'Scale' , 'Country/Series-specific Notes', 'Estimates Start After'])
index = data.index #optain all indices of data
for i in index:
    country = data.loc[i,'Country name'] #get country name from index i
    year = data.loc[i,'year'] #year from index i
    for var in weo_vars: 
        if country in common_country and year in years:
            #accessing values of variables from weo
            temp = weo.loc[(weo['Country'] == country)&(weo['Subject Descriptor'] == var) , year] 
            if temp.empty == False:
                #writing the values of variables from weo to data
                data.at[i,var] = temp[temp.index[0]]

The `data` variable now contains all information from two tables from the years 2012 - 2022 with common countries. 

Some entries of WEO report is missing and are indicated with `string` object `'na'`. 

The following is the cleaning of all invalid inputs.

In [5]:
#'na' to 'NaN' along with changing values to float of WEO variable columns
cols = data.columns.drop(['Country name', 'year'])
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')

#dataNA: with missing data for EDA
dataNa = data.copy()

#dropping nans
data = data.dropna()

#index reset
data.reset_index(drop=True, inplace=True)

#column renaming
data.columns = [ 'Country', 'Year', 'LifeLadder', 'PositiveAffect' , 'NegativeAffect',
              'GDP', 'Investment','Savings','CPI','VIGS','VIG','VEGS', 
              'VEG', 'Population', 'Revenue','Expenditure','Debt','Balance','Unemployment']

In [6]:
data.sample(5)

,Country,Year,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
54,Bahrain,2012,5.027,0.559,0.381,30.749,28.146,36.530,86.082,27.701,19.942,20.804,12.829,1.209,26.348,32.342,36.208,8.384,3.700
27,Armenia,2014,4.453,0.510,0.404,11.610,21.025,13.275,200.631,2.962,0.666,9.695,8.283,2.985,22.048,23.991,39.362,-7.750,17.600
77,Belgium,2020,6.839,0.619,0.260,524.791,24.335,25.440,108.227,-6.981,-6.981,-6.431,-6.431,11.522,49.929,58.902,112.035,1.105,5.575
353,Israel,2022,7.662,0.583,0.183,522.530,26.300,29.953,100.008,11.699,6.117,7.989,3.471,9.551,37.012,36.878,60.931,3.653,3.758
325,Iran,2016,4.653,0.592,0.526,458.042,31.245,34.135,100.008,12.393,13.639,14.414,16.447,79.926,15.265,17.038,47.904,2.890,12.425


In [7]:
data.describe()

,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
count,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000,771.000000
mean,5.968363,0.671498,0.263817,219.966346,24.441437,23.566118,135.305315,4.027429,3.690104,4.118195,3.747952,23.460494,31.389645,34.015018,53.996931,-0.948911,8.572732
std,0.893949,0.108761,0.081499,226.352593,6.680889,7.947953,93.045971,9.265745,9.087247,10.495230,9.287153,39.409880,11.722681,10.956509,31.033564,7.048594,5.804103
min,3.559000,0.297000,0.083000,1.784000,11.892000,-3.957000,40.194000,-29.230000,-25.916000,-39.078000,-29.748000,0.320000,5.117000,9.761000,0.055000,-43.825000,0.700000
25%,5.311500,0.583000,0.203000,37.499000,20.110000,18.159000,99.128000,-0.184500,-0.807000,0.092500,-0.802000,4.298500,22.350000,25.518000,35.497000,-4.571500,4.958000
50%,5.953000,0.698000,0.256000,128.814000,23.618000,23.225000,104.422000,3.937000,3.602000,3.992000,3.304000,9.051000,29.408000,33.523000,49.273000,-1.397000,6.842000
75%,6.551000,0.759000,0.314000,345.625500,27.251500,27.836000,127.940500,8.530000,8.293000,8.337500,7.708500,23.325000,40.247500,41.894500,68.989000,2.584500,10.150000
max,7.889000,0.884000,0.552000,993.681000,56.417000,58.697000,918.961000,48.744000,42.502000,84.109000,64.411000,258.497000,71.975000,60.267000,212.388000,45.462000,35.300000


# Data Analysis & Results (EDA)

In [8]:
'''
converting Year to int
DO CONVERT BACK TO STRING WHEN YEAR DOES NOT NEED TO BE INT
'''
data["Year"] = data["Year"].astype(int)

### Function
`consec_years(df,year1,year2)`
* Input: `df` as dataframe, `year1`,`year2` as int
* Output: `df_r` as dataframe
* Description: returns a copy of dataframe that includes only the countries that have consecutive years of data in range [`year1`,`year2`]

In [9]:
def consec_years(df, year1, year2):
    #containing countries with consecutive data
    curr_country = set(data[data["Year"] == year1].Country.unique().tolist())
    #increment year1
    while year1 <= year2:
        #update interscetion with current year's list
        curr_country = curr_country and set(data[data["Year"] == year1].Country.unique())
        year1+=1
    #extract names of countries to remove
    to_remove = list(set(data["Country"].unique().tolist()) - curr_country)
    #copy to preserve
    df_r = df.copy()
    #drop with names
    df_r = df_r.drop(df_r[df_r.Country.isin(to_remove)].index)
    return df_r

test_consec = consec_years(data, 2012, 2022)

In [10]:
data.sample(6)


,Country,Year,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
33,Armenia,2021,5.301,0.566,0.478,13.861,20.691,16.975,234.124,2.575,-0.680,4.150,-4.377,2.962,24.113,28.703,60.248,-3.716,15.300
444,Malta,2022,6.299,0.671,0.370,17.779,26.114,26.775,113.686,9.673,9.673,6.376,6.376,0.521,35.644,40.915,55.829,0.661,2.925
666,Sri Lanka,2013,4.365,0.776,0.208,76.934,33.249,29.949,102.233,1.426,1.426,16.207,16.207,20.585,11.605,16.609,69.457,-3.301,4.400
207,Ecuador,2017,5.840,0.793,0.314,104.296,26.280,26.104,105.442,14.805,17.120,1.569,0.717,16.777,32.179,37.965,47.030,-0.176,4.620
726,Tunisia,2021,4.499,0.451,0.336,46.688,13.973,8.016,139.554,8.448,9.547,12.380,12.380,12.019,25.675,33.293,79.942,-5.957,16.200
648,Slovenia,2016,5.937,0.597,0.272,44.754,18.427,23.204,99.944,6.289,6.583,6.163,5.716,2.064,44.244,46.163,78.520,4.776,8.025


In [15]:
test_consec.sample(6)

,Country,Year,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
457,Mongolia,2016,5.057,0.555,0.171,11.154,22.629,16.357,80.642,11.750,-5.878,33.419,32.116,3.120,24.382,39.677,95.343,-6.273,10.000
514,Norway,2015,7.603,0.796,0.209,388.160,26.908,35.913,108.587,1.883,2.713,3.926,3.011,5.184,54.191,48.203,34.334,8.994,4.531
135,Colombia,2021,5.290,0.752,0.348,318.512,18.969,13.324,108.828,26.702,17.092,15.886,0.781,51.049,27.217,34.442,64.042,-5.645,13.800
504,Nicaragua,2021,6.095,0.799,0.293,14.001,23.617,20.753,246.184,22.349,21.918,14.635,16.198,6.539,29.348,30.565,47.552,-2.865,11.100
681,Sweden,2019,7.398,0.775,0.202,533.880,25.124,30.459,106.937,2.387,-0.395,6.608,4.508,10.328,48.670,48.106,35.194,5.335,6.975
90,Bolivia,2018,5.916,0.705,0.387,40.581,20.596,16.123,101.579,2.800,2.800,4.433,4.433,11.310,28.951,37.094,53.076,-4.251,4.913


In [12]:
test_consec.head(20)

,Country,Year,LifeLadder,PositiveAffect,NegativeAffect,GDP,Investment,Savings,CPI,VIGS,VIG,VEGS,VEG,Population,Revenue,Expenditure,Debt,Balance,Unemployment
0,Albania,2012,5.510,0.553,0.271,12.324,29.824,19.674,94.783,-7.608,-5.197,-0.409,23.234,2.900,24.789,28.229,62.144,-10.202,13.400
1,Albania,2013,4.551,0.541,0.338,12.784,27.491,16.827,96.608,-0.617,-1.310,8.286,26.685,2.895,23.977,29.193,70.391,-9.253,15.900
2,Albania,2014,4.814,0.573,0.335,13.246,27.288,13.361,98.167,5.419,6.114,3.313,-6.205,2.889,26.272,31.729,71.992,-10.797,17.500
3,Albania,2015,4.607,0.579,0.350,11.389,26.237,15.804,100.000,0.072,2.161,5.254,4.292,2.881,26.440,30.758,73.716,-8.609,17.100
4,Albania,2016,4.511,0.567,0.322,11.862,25.470,16.796,101.287,8.269,9.385,10.316,-0.573,2.876,27.642,29.139,73.324,-7.572,15.200
5,Albania,2017,4.640,0.547,0.334,13.053,24.672,17.085,103.297,6.033,4.391,10.180,-1.377,2.884,27.756,29.162,71.895,-7.492,13.700
6,Albania,2018,5.004,0.592,0.319,15.157,24.118,17.109,105.393,5.767,4.235,3.468,14.145,2.883,27.457,29.077,69.472,-6.754,12.300
7,Albania,2019,4.995,0.548,0.274,15.399,23.032,14.742,106.880,4.721,3.461,1.710,-8.414,2.881,27.209,29.075,67.368,-7.571,11.470
8,Albania,2020,5.365,0.563,0.265,15.161,22.790,14.074,108.613,-19.540,-5.081,-27.569,-1.931,2.878,25.905,32.619,75.927,-8.677,11.675
9,Albania,2021,5.255,0.554,0.254,18.310,27.016,16.510,110.831,23.749,18.269,44.541,30.154,2.873,27.037,31.552,73.878,-7.661,11.400


Carry out EDA on your dataset(s); Describe in this section

In [13]:
#dataNa for NaN data analysis
print(dataNa.shape)
#data for cleaned
print(data.shape)

(1487, 19)
(771, 19)
